## Fine-Tuning and Deploying Custom Models on Amazon Bedrock

### Introduction:

This notebook will guide you through the process of creating the necessary resources and preparing the datasets for fine-tuning the Cohere  command-light-text-v14 model using Amazon Bedrock. By the end of this notebook, you will have created an IAM role, an S3 bucket, and training, validation, and testing datasets in the required format for the fine-tuning process.

## Prerequisites


- Make sure that that you have access to Cohere's command-light-text-v14. You can enable it in Amazon Bedrock. 
- Make sure that the AWS SDK is already installed and configured.

## Step 1: Install Required Libraries

Install `boto3` to build and manager resources on AWS.

In [ ]:
!pip install boto3

In [ ]:
# Importing the required libraries
import pandas as pd
import boto3
import json
import requests
import os

## Step 2: Fetch Biomedical Data from PubMed

We will define a function to retrieve abstracts from PubMed using the Entrez Programming Utilities (E-utilities). These abstracts will be used as the input data for our fine-tuning task.
PubMed is a free resource that provides access to a vast collection of biomedical literature, including abstracts and full-text articles. URL: https://www.ncbi.nlm.nih.gov/home/develop/api/

In [ ]:
def fetch_pubmed_abstracts(query, num_records):
    """
    Fetches abstracts from PubMed based on a search query.

    Args:
        query (str): Search term for PubMed.
        num_records (int): Number of abstracts to retrieve.

    Returns:
        List[dict]: A list of dictionaries with 'prompt' and 'completion' keys.
    """
    base_url = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/"
    search_url = f"{base_url}esearch.fcgi?db=pubmed&term={query}&retmax={num_records}&retmode=json"
    
    response = requests.get(search_url)
    id_list = response.json()['esearchresult']['idlist']
    
    abstracts = []
    for pubmed_id in id_list:
        fetch_url = f"{base_url}efetch.fcgi?db=pubmed&id={pubmed_id}&retmode=xml"
        fetch_response = requests.get(fetch_url)
        fetch_data = fetch_response.text
        
        # Extract abstract from XML
        start = fetch_data.find("<AbstractText>") + len("<AbstractText>")
        end = fetch_data.find("</AbstractText>")
        abstract = fetch_data[start:end]
        
        # Only add valid abstracts
        if abstract:
            abstracts.append({
                "prompt": abstract,  # Input for model fine-tuning
                "completion": "Summarized abstract"  # Placeholder for model completion task
            })
    
    return abstracts

Fetch 100 PubMed abstracts related to diabetes

In [ ]:
# Fetch 100 PubMed abstracts related to diabetes
query = "diabetes"
abstracts_data = fetch_pubmed_abstracts(query, 100)

## Step 3: Save the Dataset

We will save the fetched abstracts in JSONL format for use in the fine-tuning process.

In [ ]:
# Define the output path
abstracts_file = 'pubmed_abstracts.jsonl'
output_file_path = 'dataset/' + abstracts_file
output_dir = os.path.dirname(output_file_path)

# Ensure the directory exists
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Save the fetched abstracts to a JSONL file
try:
    with open(output_file_path, 'w') as outfile:
        for entry in abstracts_data:
            json.dump(entry, outfile)
            outfile.write('\n')
    print(f"File saved successfully to {output_file_path}")
except Exception as e:
    print(f"Error saving file: {e}")

## Step 4: Upload the Dataset to S3

We will now create an S3 bucket and upload the dataset into that bucket, which will be used in the fine-tuning job on Amazon Bedrock.

In [ ]:
# Define the file path and S3 details
bucket_name = 'bedrock-finetuning-bucket10092024'
s3_key = abstracts_file

# Specify the region
bucket_region = 'us-east-1'  # Change this if needed

# Initialize S3 client with the specified region
s3_client = boto3.client('s3', region_name=region)

# Check if the bucket exists
try:
    existing_buckets = s3_client.list_buckets()
    bucket_exists = any(bucket['Name'] == bucket_name for bucket in existing_buckets['Buckets'])

    if not bucket_exists:
        # Create the bucket based on the region
        try:
            if bucket_region == 'us-east-1':
                # For us-east-1, do not specify LocationConstraint
                s3_client.create_bucket(Bucket=bucket_name)
                print(f"Bucket {bucket_name} created successfully in us-east-1.")
            else:
                # For other regions, specify the LocationConstraint
                s3_client.create_bucket(
                    Bucket=bucket_name,
                    CreateBucketConfiguration={'LocationConstraint': bucket_region}
                )
                print(f"Bucket {bucket_name} created successfully in {bucket_region}.")
        except Exception as e:
            print(f"Error creating bucket: {e}")
            raise e
    else:
        print(f"Bucket {bucket_name} already exists.")

    # Upload the file to S3
    try:
        s3_client.upload_file(output_file_path, bucket_name, s3_key)
        print(f"File uploaded to s3://{bucket_name}/{s3_key}")
    except Exception as e:
        print(f"Error uploading to S3: {e}")

except Exception as e:
    print(f"Error: {e}")

## Step 5: Fine-Tune the Model on Amazon Bedrock

Now, we create a fine-tuning job using Amazon Bedrock's API. We'll first list the available foundation models.

In [ ]:
# Specify the AWS region and initialize the Bedrock client
bedrock_region = "us-east-1"

# Initialize the Bedrock client
bedrock = boto3.client(service_name="bedrock", region_name=bedrock_region)

# List foundation models available for fine-tuning
response = bedrock.list_foundation_models(byCustomizationType="FINE_TUNING")

# Display the available models
for model in response["modelSummaries"]:
    print(f"Model ID: {model['modelId']}")
    print(f"Model Name: {model['modelName']}")
    print("-----")

## Step 6: Create an IAM Role for Fine-Tuning

We create an IAM role that Bedrock can assume to access S3 during the fine-tuning process.

In [ ]:
# Initialize IAM client
iam_client = boto3.client('iam')

# Define role name and trust policy
role_name = "BedrockFineTuningRole"
role_description = "Role for Bedrock fine-tuning job"

trust_policy = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Principal": {
                "Service": "bedrock.amazonaws.com"
            },
            "Action": "sts:AssumeRole",
        }
    ]
}

# Create the IAM role
try:
    response = iam_client.create_role(
        RoleName=role_name,
        AssumeRolePolicyDocument=json.dumps(trust_policy),
        Description=role_description
    )
    role_arn = response['Role']['Arn']
    print(f"Created role with ARN: {role_arn}")
except Exception as e:
    print(f"Error creating role: {e}")

# Attach permission policies to allow access to S3
permission_policy = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Action": [
                "s3:GetObject",
                "s3:PutObject",
                "s3:ListBucket"
            ],
            "Resource": [
                f"arn:aws:s3:::{bucket_name}",
                f"arn:aws:s3:::{bucket_name}/*"
            ]
        }
    ]
}

iam_client.put_role_policy(
    RoleName=role_name,
    PolicyName="BedrockFineTuningS3Policy",
    PolicyDocument=json.dumps(permission_policy)
)

## Step 7: Submit the Fine-Tuning Job

We submit the fine-tuning job with your custom model name and configuration.

In [ ]:
# Define the job parameters
base_model_id = "cohere.command-light-text-v14:7:4k"
job_name = "cohere-Summarizer-medical-finetuning-job-v1"
model_name = "cohere-Summarizer-medical-Tuned-v1"

# Submit the fine-tuning job
bedrock.create_model_customization_job(
    customizationType="FINE_TUNING",
    jobName=job_name,
    customModelName=model_name,
    roleArn=role_arn,
    baseModelIdentifier=base_model_id,
    hyperParameters={
        "epochCount": "1", # 3 Adjust based on convergence and overfitting
        "batchSize": "8", # 16 Adjust based on memory availability and training speed
        "learningRate": "0.00001", # Adjust based on training stability and speed
    },
    trainingDataConfig={"s3Uri": f"s3://{bucket_name}/{s3_key}"},
    outputDataConfig={"s3Uri": f"s3://{bucket_name}/finetuned/"}
)

You can check the job status (next step) to make sure if it is finished or still being trained.

## Step 8: Monitor the Job Status

We monitor the fine-tuning job status to check its progress.

In [ ]:
# Check the job status
status = bedrock.get_model_customization_job(jobIdentifier=job_name)["status"]
print(f"Job status: {status}")

## Step 9: Perform Model Inference

To use the model for inference, you need to purchase "Provisioned Throughput." On Amazon Bedrock sidebar in your AWS console, go to "Custom Models" and then choose the "Models" tab, select the model you have trained, and then click on "Purchase Provisioned Throughput." Give the provisioned throughput a name, select a commitment term (you can choose "No Commitment" for testing), and then click "Purchase Provisioned Throughput." Once this is set up, you'll be able to use the model for inference.

On Amazon Bedrock sidebar in your AWS console, go to "Custom Models" and then choose the "Jobs" tab. Here you cann see the status of the training job. Once the training job is finishedn the status will be changed to "Complete".

![Alt text](images/job-status.png)

Once the job is "Complete", go to "Custom Models" and then choose the "Models" tab, select the model you have trained, and then click on "Purchase Provisioned Throughput."

![Alt text](images/models.png)

Give the provisioned throughput a name, select a commitment term (you can choose "No Commitment" for testing), and then click "Purchase Provisioned Throughput." Once this is set up, you'll be able to use the model for inference. You can also see the estimated price for each commitment term.

![Alt text](images/provision.png)

To access your deployed model endpoint, you need the model ARN. Navigate to "Custom Models," then select the "Models" tab. Choose the model you’ve trained and copy the "Model ARN" for use in the next step.

![Alt text](images/model-arn.png)

![Alt text](images/in-service.png)

In [ ]:
# Initialize Bedrock runtime client
bedrock_runtime = boto3.client(service_name="bedrock-runtime", region_name=bedrock_region)

# Define a prompt for model inference
prompt = """
Summarize the following medical abstract:
This study investigates the impact of diabetes on cardiovascular health. The research involved a cohort of 200 patients 
with type 2 diabetes, tracking their cardiovascular events over a period of 5 years. The results indicate a significant 
correlation between diabetes duration and the incidence of heart disease.
"""

# Define the inference request body
body = {
    "prompt": prompt,
    "temperature": 0.5,
    "p": 0.9,
    "max_tokens": 80,
}

# Specify the ARN of the custom model
custom_model_arn = "YOUR_MODEL_ARN" #Put your model ARN here

# Invoke the custom model for inference
try:
    response = bedrock_runtime.invoke_model(
        modelId=custom_model_arn,
        body=json.dumps(body)
    )

    # Read and parse the response
    response_body = response['body'].read().decode('utf-8')
    result = json.loads(response_body)

    # Extract the summary from the response
    summary_text = result['generations'][0]['text']
    print("Extracted Summary:", summary_text)
except Exception as e:
    print(f"Error invoking model: {e}")

You can also use the Playground to test your fine-tuned model. To do this, go to the **Test** section under **Playground** in the Amazon Bedrock console. Select your fine-tuned model, then enter your prompt to start testing.

![Alt text](images/playground.png)

### Cleanup

<span style="color:red">To avoid incurring additional costs, please ensure that you **remove any provisioned throughput**.</span>

<span style="color:red">You can remove provisioned throughput by navigating to the **Provisioned Throughput** section from the sidebar in the Amazon Bedrock console. Select the active provisioned throughput and delete it.</span>